# Demo: Creating Content from Webpage

**Objective:**

This notebook demoes how a simple workflow of sequential requests to large language models can be leveraged to create multiple pieces of content from an arbitrary webpage.

This workflow consists of three sequential steps:
1. Generate a summary of desired webpage
2. Structure the summary according to a user-defined outline.
3. Create a social media post from structured summary with custom persona.

**Requirements:**

This demo uses the OpenAI Chat API and, thus, requires an OpenAI API key.

**Disclaimer:**

This notebook is strictly for educational purposes. Users should employ it responsibly, ensuring accuracy, respecting privacy and copyright, and avoiding the dissemination of misinformation. Always consider the ethical implications and context of the generated content.

## Setup: Load and parse HTML webpage

Before we can start our workflow, we extract the text content from the desired webpage, focusing on paragraphs and headers, to use downstream. We use the Llama 2 announcement from Meta AI in this demo.

In [3]:
# Extract text from webpage

from IPython.display import Markdown
from genaipy.extractors.web import extract_tags_contents

URL = "https://ai.meta.com/blog/large-language-model-llama-meta-ai/"
TAGS = ["p", "h1"]

text = extract_tags_contents(url=URL, tags=TAGS)
Markdown(text[:1200]) # Display first 1200 characters

Introducing LLaMA  A foundational  65 billion parameter large language model

February 24  2023

UPDATE  We just launched Llama 2   for more information on the latest see our blog post on Llama 2  

As part of Meta s commitment to open science  today we are publicly releasing LLaMA  Large Language Model Meta AI   a state of the art foundational large language model designed to help researchers advance their work in this subfield of AI  Smaller  more performant models such as LLaMA enable others in the research community who don t have access to large amounts of infrastructure to study these models  further democratizing access in this important  fast changing field 

Training smaller foundation models like LLaMA is desirable in the large language model space because it requires far less computing power and resources to test new approaches  validate others  work  and explore new use cases  Foundation models train on a large set of unlabeled data  which makes them ideal for fine tuning for a variety of tasks  We are making LLaMA available at several sizes  7B  13B  33B  and 65B parameters  and also sharing a LLaMA model card that details how we built the model in keeping with our app

## Step [1/3]: Generate Summary

In the first step of the workflow, we provide the input text sample to `GPT-3.5-turbo` and request it to generate a 150 words long summary. This step serves as the foundation for the following steps.

In [4]:

from genaipy.openai_apis.chat import get_chat_response
from genaipy.prompts.build_prompt import build_prompt
from genaipy.prompts.generate_summaries import SUMMARY_PROMPT_TPL

prompt = build_prompt(template=SUMMARY_PROMPT_TPL, text=text, max_words=150)

summary = get_chat_response(prompt=prompt, temperature=0.2)
Markdown(summary)

Meta has publicly released LLaMA, a large language model designed to help researchers in the field of AI. Smaller models like LLaMA require less computing power and resources, making them more accessible for studying and fine-tuning. LLaMA is available in different sizes, ranging from 7B to 65B parameters. Large language models have shown significant potential in various tasks, but limited access to them hinders progress in understanding their workings and addressing issues like bias and misinformation. LLaMA is trained on a large dataset and can be applied to different use cases. The model is released under a noncommercial license for research purposes, and access is granted on a case-by-case basis. Collaboration among the AI community is encouraged to develop responsible guidelines for large language models.

## Step [2/3]: Structure Summary with Outline

In the second step of the workflow, we use the generated summary and utilize `GPT-3.5-turbo` to structure it according to a custom outline. Notably, this structured outline is completely customizable to the requirements of the use case (e.g., meeting minutes, reports, etc.).

Here, we opted for a basic outline for structuring summaries displayed below. Since the structured summary is in `Markdown`, it can be directly saved and displayed in many text editors.

In [5]:
# Load structured outline

from genaipy.outlines.basic_summary_outline import BASIC_SUMMARY_OUTLINE

tpl = BASIC_SUMMARY_OUTLINE.strip()
print(tpl)

# [Short Title]

## Summary
(Exactly 1 sentence long summary of the main topic and objective.)

## Key points 
(bulleted list of 3-5 main points.)
- [Key insight 1]
- [Key insight 2]
- [Key insight 3]
- ... (only if needed)

## Key concepts
(bulleted list of exactly 3-5 main concepts and entities.)
- [Key concept/entity 1]
- [Key concept/entity 2]
- [Key concept/entity 3]
- ... (only if needed)


In [6]:
# Generate structured summary

from genaipy.prompts.style_texts import STYLE_TEXT_TPL

style_prompt = build_prompt(template=STYLE_TEXT_TPL, text=summary, tpl=tpl)

styled_summary = get_chat_response(prompt=style_prompt, temperature=0) 
Markdown(styled_summary)

```
# LLaMA: A Large Language Model for AI Research

## Summary
LLaMA is a large language model developed by Meta to assist researchers in the field of AI by providing a more accessible and resource-efficient option for studying and fine-tuning language models.

## Key points
- LLaMA is a smaller language model that requires less computing power and resources, making it easier for researchers to work with.
- The model is available in different sizes, ranging from 7B to 65B parameters, allowing researchers to choose the appropriate scale for their needs.
- Limited access to large language models has hindered progress in understanding their workings, addressing bias, and combating misinformation.
- LLaMA is trained on a large dataset and can be applied to various use cases, enhancing its versatility and applicability.
- The model is released under a noncommercial license for research purposes, and access is granted on a case-by-case basis.

## Key concepts
- Large language models: Advanced AI models designed to process and generate human-like text.
- Accessibility: Making resources and tools more available and easier to use for researchers.
- Bias and misinformation: Issues related to the presence of prejudices and false information in language models and their outputs.
```

## Step [3/3]: Generate Social Media Post

In the final step of the workflow, we generate a social media post from the structured summary using `GPT-4-turbo`. 
Notably, we set a custom role for the LLM via the system message to tailor the style and tone of the post.

In this case, we set the LLM's role to a  professional content creator on Linkedin excelling at communicating complex AI topics in a simple way. Feel free to change the system message below, to explore different types of social media posts!

In [8]:
# Define custom role

system_message = "You are an expert content creator on LinkedIn. You excell at communicating complex topics in Generative AI in a simple and professional way."
Markdown(system_message)

You are an expert content creator on LinkedIn. You excell at communicating complex topics in Generative AI in a simple and professional way.

In [9]:
# Generate LinkedIn post

from genaipy.prompts.generate_posts import GENERATE_POST_TPL

generate_prompt = build_prompt(template=GENERATE_POST_TPL, text=styled_summary, max_words=150)

post = get_chat_response(
    prompt=generate_prompt,
    sys_message=system_message,
    temperature=0.5,
    model="gpt-4-1106-preview"
    )

Markdown(post)

🚀 Thrilled to share insights on Meta's LLaMA, a game-changer in AI research!

🔍 **Understanding LLaMA:**
As a passionate advocate for democratizing AI, I'm excited about LLaMA's potential. It's a lean, mean, AI research machine, tailored for accessibility and efficiency.

🌟 **Why LLaMA Stands Out:**
- Scaled for simplicity: With models from 7B to 65B parameters, LLaMA is fit for various research scopes.
- Democratizing AI: Finally, a language model that broadens research horizons without the computational cost barrier.
- A step towards transparency: By addressing AI biases and misinformation, LLaMA paves the way for more ethical AI.

🔐 **Access & Use:**
Leveraging LLaMA is a breeze under its noncommercial research license, though it's a case-by-case deal.

Let's embrace this stride in AI—more accessible, less resource-heavy, and a catalyst for innovation! 🌐💡 #LLaMA #AILanguageModel #AIResearch #Innovation